In [13]:
import glob
import numpy as np
import pickle
import open3d as o3d
import torch

def translate_boxes_to_open3d_instance(gt_boxes):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])
    rot = o3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = o3d.geometry.OrientedBoundingBox(center, rot, lwh)

    line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    # import ipdb; ipdb.set_trace(context=20)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = o3d.utility.Vector2iVector(lines)

    return line_set, box3d

def draw_box(gt_boxes, color=(0, 1, 0), ref_labels=None, score=None):
    line_sets = []
    for i in range(gt_boxes.shape[0]):
        line_set, box3d = translate_boxes_to_open3d_instance(gt_boxes[i])
        if ref_labels is None:
            line_set.paint_uniform_color(color)
        else:
            line_set.paint_uniform_color(box_colormap[ref_labels[i]])
        line_sets.append(line_set)
    return line_sets


def visualize_pc(new_pc_rect, label=None, bbox=None):
    if np.random.random() < 1:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(new_pc_rect[:, :3])
        axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
        
        if label is not None:
            pc_colors = np.repeat([0, 0, 1], len(new_pc_rect)).reshape(3, len(new_pc_rect)).T
            pc_colors[np.argwhere(label==1)] = [1, 0, 0]
            print("pc_colors", pc_colors.shape)
            pcd.colors = o3d.utility.Vector3dVector(pc_colors)
            
        if bbox is not None:
            vis = draw_box(bbox, (0, 1, 0))
            o3d.visualization.draw_geometries([pcd, axes]+vis)
        else:
            o3d.visualization.draw_geometries([pcd, axes])

In [4]:
%ls /mnt/wato-drive/perception_pcds/road/npy

1000.npy  1642.npy  2284.npy  2926.npy  3568.npy  4201.npy  4785.npy  5369.npy
1001.npy  1643.npy  2285.npy  2927.npy  3569.npy  4202.npy  4786.npy  536.npy
1002.npy  1644.npy  2286.npy  2928.npy  3570.npy  4203.npy  4787.npy  5370.npy
1003.npy  1645.npy  2287.npy  2929.npy  3571.npy  4204.npy  4788.npy  5371.npy
1004.npy  1646.npy  2288.npy  2930.npy  3572.npy  4205.npy  4789.npy  5372.npy
1005.npy  1647.npy  2289.npy  2931.npy  3573.npy  4206.npy  478.npy   5373.npy
1006.npy  1648.npy  2290.npy  2932.npy  3574.npy  4207.npy  4790.npy  5374.npy
1007.npy  1649.npy  2291.npy  2933.npy  3575.npy  4208.npy  4791.npy  5375.npy
1008.npy  1650.npy  2292.npy  2934.npy  3576.npy  4209.npy  4792.npy  5376.npy
1009.npy  1651.npy  2293.npy  2935.npy  3577.npy  420.npy   4793.npy  5377.npy
1010.npy  1652.npy  2294.npy  2936.npy  3578.npy  4210.npy  4794.npy  5378.npy
1011.npy  1653.npy  2295.npy  2937.npy  3579.npy  4211.npy  4795.npy  5379.npy
1012.npy  1654.npy  2296.npy  2938.npy  3580.npy  421

In [9]:
pred_dict = "/home/ch35li/alvin/PointPillars/NewST3D/OpenPCDet/output/da-nuscenes-wato_models/secondiou_st3d/secondiou_st3d/pickled_dicts/bbox_416.pickle"
pc_npy = "/mnt/wato-drive/perception_pcds/road/npy/416.npy"

with open(pred_dict, "rb") as f:
    bbox_dict = pickle.load(f)
a = np.load(pc_npy)

In [10]:
bbox_dict

([{'pred_boxes': tensor([], device='cuda:0', size=(0, 7)),
   'pred_scores': tensor([], device='cuda:0'),
   'pred_labels': tensor([], device='cuda:0', dtype=torch.int64),
   'pred_cls_scores': tensor([], device='cuda:0'),
   'pred_iou_scores': tensor([], device='cuda:0')}],
 {})

In [14]:
# bbox = np.stack([box for i, box in enumerate(bbox_dict[0]["pred_boxes"].cpu().numpy()) if bbox_dict[0]["pred_cls_scores"].cpu().numpy()[i] > 0.3], axis=0)
visualize_pc(a, bbox=None)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [15]:
pred_ret_targ = "/home/ch35li/alvin/PointPillars/NewST3D/OpenPCDet/output/da-nuscenes-wato_models/secondiou_st3d/secondiou_st3d/monke/pred_ret_targ_ep6_5000.pkl"

with open(pred_ret_targ, "rb") as f:
    pred, ret, targ = pickle.load(f)

In [20]:
targ["voxels"].size()

torch.Size([18884, 5, 3])

In [22]:
%ls /mnt/wato-drive/nuScenes/v1.0-trainval/

gt_database/                            v1.0-trainval01_blobs.tgz
gt_database_10sweeps_withvelo/          v1.0-trainval02_blobs.tgz
infos_train.pkl                         v1.0-trainval03_blobs.tgz
infos_val.pkl                           v1.0-trainval04_blobs.tgz
kitti_dbinfos_train.pkl                 v1.0-trainval05_blobs.tgz
maps/                                   v1.0-trainval06_blobs.tgz
nuscenes_dbinfos_10sweeps_withvelo.pkl  v1.0-trainval07_blobs.tgz
nuscenes_infos_10sweeps_train.pkl       v1.0-trainval08_blobs.tgz
nuscenes_infos_10sweeps_val.pkl         v1.0-trainval09_blobs.tgz
samples/                                v1.0-trainval10_blobs.tgz
sweeps/                                 v1.0-trainval_meta.tgz
v1.0-trainval/


In [23]:
dbinfor = "/mnt/wato-drive/nuScenes/v1.0-trainval/nuscenes_dbinfos_10sweeps_withvelo.pkl"

with open(dbinfor, "rb") as f:
    all_db_info = pickle.load(f)

In [28]:
one_batch_path = "/home/ch35li/alvin/PointPillars/NewST3D/OpenPCDet/output/test.pkl"

with open(one_batch_path, "rb") as f:
    one_batch = pickle.load(f)

In [35]:
with open("/home/ch35li/alvin/PointPillars/NewST3D/OpenPCDet/output/da-nuscenes-kitti_models/secondiou_st3d/secondiou_st3d/monke/pred_ret_targ_ep2_500.pkl", "rb") as f:
    pred_dicts, ret_dict, target_batch = pickle.load(f)

print(pred_dicts[0].keys())
print(target_batch)
a = target_batch["points"].cpu().numpy()[:, 1:]

bbox = np.stack([box.cpu().numpy() for i, box in enumerate(ps_labels[0][0]["pred_boxes"]) if ps_labels[0][0]["pred_scores"][i] > 0.3], axis=0)
visualize_pc(a, bbox=bbox)

ModuleNotFoundError: No module named 'pcdet'